# IF3170 Artificial Intelligence | Tugas Besar 2

Nama Kelompok: EgiluyIzinnn🙏


Group Number: 34

Group Members:
- Nicholas Andhika Lucas (13523014)
- Shanice Feodora Tjahjono (13523097)
- Jonathan Kenan Budianto (13523139)
- Mahesa Fadhillah Andre (13523140)
- Muhammad Farrel Wibowo (13523153)

## Import Libraries

In [67]:
%pip install pandas numpy matplotlib seaborn scikit-learn imblearn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif, VarianceThreshold
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE

## Import Dataset

In [42]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public

df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

Train shape: (3096, 38)
Test shape: (1328, 37)


# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [43]:
print("EXPLORATORY DATA ANALYSIS")

# Dataset Information
print("\nInformasi Dataset:") 
df_train.info()

# Ringkasan Statistik
print("\nRingkasan Statistik:")
print(df_train.describe())

# Cek missing values
print("\nMissing Values:")
print(df_train.isnull().sum())
print(f"Total missing values: {df_train.isnull().sum().sum()}")

# Cek duplikasi
print(f"\nDuplicate rows: {df_train.duplicated().sum()}")

# Target Distribution
print("\nTarget Distribution:")
print(df_train['Target'].value_counts())
print("\nLabel Distribution (%):")
print(df_train['Target'].value_counts(normalize=True) * 100)

# Visualisasi Data
# TODO


EXPLORATORY DATA ANALYSIS

Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Student_ID                                      3096 non-null   int64  
 1   Marital status                                  3096 non-null   int64  
 2   Application mode                                3096 non-null   int64  
 3   Application order                               3096 non-null   int64  
 4   Course                                          3096 non-null   int64  
 5   Daytime/evening attendance	                     3096 non-null   int64  
 6   Previous qualification                          3096 non-null   int64  
 7   Previous qualification (grade)                  3096 non-null   float64
 8   Nacionality                                     3096 non-null   int64  


# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [44]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.

# Training and validation split on 80/20 ratio
train_set, val_set = train_test_split(
    df_train, 
    test_size=0.2, 
    random_state=42, 
    stratify=df_train['Target']
)

print("\nTarget Distribution:")
print("\nFull Training Data:")
print(df_train['Target'].value_counts())
print("\nTraining Set (80%):")
print(train_set['Target'].value_counts())
print("\nValidation Set (20%):")
print(val_set['Target'].value_counts())


Target Distribution:

Full Training Data:
Target
Graduate    1546
Dropout      994
Enrolled     556
Name: count, dtype: int64

Training Set (80%):
Target
Graduate    1236
Dropout      795
Enrolled     445
Name: count, dtype: int64

Validation Set (20%):
Target
Graduate    310
Dropout     199
Enrolled    111
Name: count, dtype: int64


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [45]:
# There is no missing data in EDA, skip this process

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [46]:
numerical_cols = train_set.select_dtypes(include=['number']).columns

# Exclude ID, target and categorical/binary columns
exclude_cols = [
    'Student_ID', 'Target',
    'Marital status', 'Application mode', 'Application order', 'Course',
    'Daytime/evening attendance\t', 'Previous qualification', 'Nacionality',
    "Mother's qualification", "Father's qualification",
    "Mother's occupation", "Father's occupation",
    'Displaced', 'Educational special needs', 'Debtor',
    'Tuition fees up to date', 'Gender', 'Scholarship holder', 'International'
]

cols_to_process = [c for c in numerical_cols if c not in exclude_cols]

# Check for outliers using IQR
print("\nOutlier Detection with IQR Method:")

bounds = {}
log_transform_cols = []

for col in cols_to_process:
    Q1 = train_set[col].quantile(0.25)
    Q3 = train_set[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers_count = ((train_set[col] < lower) | (train_set[col] > upper)).sum()
    outliers_percent = outliers_count / len(train_set) * 100

    if outliers_count > 0:
        print(f"Column {col}: {outliers_count} outliers ({outliers_percent:.2f}%)")

    if 'grade' in col.lower():
        continue

    if outliers_percent > 10:
        log_transform_cols.append(col)
    elif outliers_percent > 0:
        bounds[col] = (lower, upper)

# Apply log transform
for col in log_transform_cols:
    train_set[col] = train_set[col].apply(lambda x: np.log1p(x) if x > 0 else x)
    val_set[col]   = val_set[col].apply(lambda x: np.log1p(x) if x > 0 else x)

# Apply capping
for col, (lower, upper) in bounds.items():
    train_set[col] = np.clip(train_set[col], lower, upper)
    val_set[col]   = np.clip(val_set[col],   lower, upper)

print("\nOutlier Handling Complete")



Outlier Detection with IQR Method:
Column Previous qualification (grade): 95 outliers (3.84%)
Column Admission grade: 46 outliers (1.86%)
Column Age at enrollment: 251 outliers (10.14%)
Column Curricular units 1st sem (credited): 329 outliers (13.29%)
Column Curricular units 1st sem (enrolled): 246 outliers (9.94%)
Column Curricular units 1st sem (evaluations): 87 outliers (3.51%)
Column Curricular units 1st sem (approved): 99 outliers (4.00%)
Column Curricular units 1st sem (grade): 406 outliers (16.40%)
Column Curricular units 1st sem (without evaluations): 174 outliers (7.03%)
Column Curricular units 2nd sem (credited): 301 outliers (12.16%)
Column Curricular units 2nd sem (enrolled): 215 outliers (8.68%)
Column Curricular units 2nd sem (evaluations): 59 outliers (2.38%)
Column Curricular units 2nd sem (approved): 27 outliers (1.09%)
Column Curricular units 2nd sem (grade): 494 outliers (19.95%)
Column Curricular units 2nd sem (without evaluations): 153 outliers (6.18%)

Outlier Ha

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [47]:
# There is no duplicate data in EDA, skip this process

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [48]:
print("*** Applying Feature Engineering ***")

# *** 1. INTERACTION FEATURES ***
print("\n1. Creating Interaction Features")
def interaction_features(data):
    df_eng = data.copy()
    
    # Total units (Sum)
    df_eng['Total_enrolled'] = df_eng['Curricular units 1st sem (enrolled)'] + df_eng['Curricular units 2nd sem (enrolled)']
    df_eng['Total_approved'] = df_eng['Curricular units 1st sem (approved)'] + df_eng['Curricular units 2nd sem (approved)']
    df_eng['Total_credited'] = df_eng['Curricular units 1st sem (credited)'] + df_eng['Curricular units 2nd sem (credited)']
    print("Created Total_enrolled, Total_approved, Total_credited features.")
    
    # Average Grade
    df_eng['Average_grade'] = (df_eng['Curricular units 1st sem (grade)'] + df_eng['Curricular units 2nd sem (grade)']) / 2
    print("Created Average_grade feature.")
    
    # Approval Rates
    df_eng['Approval_rate_sem1'] = np.where(
        df_eng['Curricular units 1st sem (enrolled)'] > 0,
        df_eng['Curricular units 1st sem (approved)'] / df_eng['Curricular units 1st sem (enrolled)'], 
        0
    )
    df_eng['Approval_rate_sem2'] = np.where(
        df_eng['Curricular units 2nd sem (enrolled)'] > 0, 
        df_eng['Curricular units 2nd sem (approved)'] / df_eng['Curricular units 2nd sem (enrolled)'], 
        0
    )
    print("Created Approval_rate_sem1 and Approval_rate_sem2 features.")
    
    # Grade Trend
    df_eng['Grade_improvement'] = df_eng['Curricular units 2nd sem (grade)'] - df_eng['Curricular units 1st sem (grade)']
    print("Created Grade_improvement feature.")
    
    return df_eng

train_set_eng = interaction_features(train_set)
val_set_eng = interaction_features(val_set)

*** Applying Feature Engineering ***

1. Creating Interaction Features
Created Total_enrolled, Total_approved, Total_credited features.
Created Average_grade feature.
Created Approval_rate_sem1 and Approval_rate_sem2 features.
Created Grade_improvement feature.
Created Total_enrolled, Total_approved, Total_credited features.
Created Average_grade feature.
Created Approval_rate_sem1 and Approval_rate_sem2 features.
Created Grade_improvement feature.
Created Total_enrolled, Total_approved, Total_credited features.
Created Average_grade feature.
Created Approval_rate_sem1 and Approval_rate_sem2 features.
Created Grade_improvement feature.
Created Total_enrolled, Total_approved, Total_credited features.
Created Average_grade feature.
Created Approval_rate_sem1 and Approval_rate_sem2 features.
Created Grade_improvement feature.


In [49]:
# *** 2. FEATURE SELECTION ***
print("\n2. Selecting and Dropping Features")

analysis_df = train_set_eng.copy()
cat_cols = [
    'Marital status', 'Application mode', 'Application order', 'Course',
    'Daytime/evening attendance\t', 'Previous qualification', 'Nacionality',
    "Mother's qualification", "Father's qualification", "Mother's occupation",
    "Father's occupation", 'Displaced', 'Educational special needs', 'Debtor',
    'Tuition fees up to date', 'Gender', 'Scholarship holder', 'International'
]

# Label Encode Categoricals and Target
le = LabelEncoder()
for col in cat_cols:
    if col in analysis_df.columns:
        analysis_df[col] = le.fit_transform(analysis_df[col])

analysis_df['Target_Encoded'] = le.fit_transform(analysis_df['Target'])
analysis_df = analysis_df.drop(columns=['Target'])

# Redundancy Filtering (Correlation)
# We look for features correlated > 0.90 with each other.
# We drop the one that is LESS correlated with the Target.

# 1. Calculate Feature-Feature Correlation
feature_corr = analysis_df.drop(columns=['Target_Encoded']).corr().abs()

# 2. Calculate Feature-Target Correlation
target_corr = analysis_df.drop(columns=['Target_Encoded']).apply(
    lambda x: x.corr(analysis_df['Target_Encoded'])
).abs()

# 3. Identify pairs and select the one to drop
upper = feature_corr.where(np.triu(np.ones(feature_corr.shape), k=1).astype(bool))
features_to_drop = []

for column in upper.columns:
    if any(upper[column] > 0.90):
        # Find all features this column conflicts with
        conflicting_features = upper.index[upper[column] > 0.90].tolist()
        for feature in conflicting_features:
            # Compare their correlation with the Target
            score_col = target_corr[column]
            score_feat = target_corr[feature]
            
            # Drop the weaker predictor
            if score_col < score_feat:
                features_to_drop.append(column)
            else:
                features_to_drop.append(feature)

features_to_drop = list(set(features_to_drop))

print(f"Dropping {len(features_to_drop)} redundant features: {features_to_drop}")
train_set_eng = train_set_eng.drop(columns=features_to_drop)
val_set_eng = val_set_eng.drop(columns=features_to_drop)

print("\nRemaining features: ")
print(f"Count: {len(train_set_eng.columns)} - List: {train_set_eng.columns.tolist()}")

# NOTE: Random Forest Important is tricky and may be tested later after first training

# # --- STEP B: Random Forest Importance ---
# # Train a Random Forest on the REMAINING features to find the most useful ones
# rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# # Prepare X and y for RF
# X_rf = analysis_df.drop(columns=['Target_Encoded'] + features_to_drop)
# y_rf = analysis_df['Target_Encoded']

# rf.fit(X_rf, y_rf)

# # Rank Features
# importances = pd.DataFrame({
#     'Feature': X_rf.columns,
#     'Importance': rf.feature_importances_
# }).sort_values(by='Importance', ascending=False)

# # Select Top 25
# final_features = importances.head(25)['Feature'].tolist()

# print("\nTop 10 Features:")
# print(importances.head(10))
# print(f"\nFinal Selected Features ({len(final_features)}): {final_features}")


2. Selecting and Dropping Features
Dropping 10 redundant features: ['Curricular units 1st sem (approved)', 'Nacionality', 'Curricular units 1st sem (enrolled)', 'Average_grade', 'Curricular units 1st sem (credited)', 'Total_credited', 'Total_enrolled', 'Curricular units 2nd sem (approved)', 'Total_approved', 'Curricular units 1st sem (grade)']

Remaining features: 
Count: 35 - List: ['Student_ID', 'Marital status', 'Application mode', 'Application order', 'Course', 'Daytime/evening attendance\t', 'Previous qualification', 'Previous qualification (grade)', "Mother's qualification", "Father's qualification", "Mother's occupation", "Father's occupation", 'Admission grade', 'Displaced', 'Educational special needs', 'Debtor', 'Tuition fees up to date', 'Gender', 'Scholarship holder', 'Age at enrollment', 'International', 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (without evaluations)', 'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', '

c:\Users\reala\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\reala\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [50]:
# *** 3. BINNING ***
print("\n*** BINNING FEATURES ***")

# 1. Age at Enrollment
# Age was log-transformed during outlier handling, so use log-scale thresholds
print("\n1. Age at Enrollment Binning (Domain-Based)")

def bin_age(age_log):
    if age_log < np.log(20):
        return 0  # Young
    elif age_log <= np.log(25):
        return 1  # Traditional
    else:
        return 2  # Mature

train_set_eng['Age at enrollment'] = train_set_eng['Age at enrollment'].apply(bin_age)
val_set_eng['Age at enrollment'] = val_set_eng['Age at enrollment'].apply(bin_age)

print("   Update Age at enrollment feature (0=Young <20, 1=Traditional 20-25, 2=Mature >25)")
print("   Age at enrollment distribution:\n")
print(train_set_eng['Age at enrollment'].value_counts().sort_index())


*** BINNING FEATURES ***

1. Age at Enrollment Binning (Domain-Based)
   Update Age at enrollment feature (0=Young <20, 1=Traditional 20-25, 2=Mature >25)
   Age at enrollment distribution:

Age at enrollment
0     591
1    1256
2     629
Name: count, dtype: int64


In [51]:
# *** FREQUENCY ANALYSIS FOR BINNING ***
print("\n*** Analyzing Frequency Distribution for Binning Candidates ***")

binning_candidates = ["Mother's occupation", "Father's occupation", "Previous qualification"]

for col in binning_candidates:
    print(f"\n{'='*60}")
    print(f"Column: {col}")
    print(f"{'='*60}")
    
    # Value counts and percentages
    value_counts = train_set_eng[col].value_counts()
    value_percentages = train_set_eng[col].value_counts(normalize=True) * 100
    
    # Summary statistics
    print(f"Total unique categories: {train_set_eng[col].nunique()}")
    print(f"Total samples: {len(train_set_eng)}")
    print(f"\nTop 10 most frequent categories:")
    print(pd.DataFrame({
        'Count': value_counts.head(10),
        'Percentage': value_percentages.head(10).round(2)
    }))
    
    # Low frequency analysis
    low_freq_threshold_5pct = len(train_set_eng) * 0.05  # 5% threshold
    low_freq_5pct = (value_counts < low_freq_threshold_5pct).sum()
    print(f"Categories with < 5% frequency: {low_freq_5pct}/{train_set_eng[col].nunique()}")


*** Analyzing Frequency Distribution for Binning Candidates ***

Column: Mother's occupation


Total unique categories: 30
Total samples: 2476

Top 10 most frequent categories:
                     Count  Percentage
Mother's occupation                   
9                      858       34.65
4                      474       19.14
5                      300       12.12
3                      184        7.43
2                      179        7.23
7                      166        6.70
0                       85        3.43
6                       57        2.30
1                       55        2.22
90                      40        1.62
Categories with < 5% frequency: 24/30

Column: Father's occupation
Total unique categories: 39
Total samples: 2476

Top 10 most frequent categories:
                     Count  Percentage
Father's occupation                   
9                      545       22.01
7                      361       14.58
5                      286       11.55
4                      229        9.25
3                      220        8.89
8                      188  

In [52]:
# *** APPLY FREQUENCY-BASED BINNING ***
# Group low frequency categories (<5% threshold) into 'Other' category (-1 in numeric)
print("\n*** Applying Frequency-Based Binning ***")

def apply_frequency_binning(train_set, val_set, column_name, threshold_pct=5.0, other_value=-1, verbose=True):
    min_count = len(train_set) * (threshold_pct / 100) # Threshold
    value_counts = train_set[column_name].value_counts()
    categories_to_bin = value_counts[value_counts < min_count].index.tolist()
    original_unique = train_set[column_name].nunique()
    
    # Apply bin
    train_set[column_name] = train_set[column_name].apply(
        lambda x: other_value if x in categories_to_bin else x
    )
    val_set[column_name] = val_set[column_name].apply(
        lambda x: other_value if x in categories_to_bin else x
    )
    
    other_count = (train_set[column_name] == other_value).sum()
    print(f"   Binned {len(categories_to_bin)}/{original_unique} categories to 'Other'")
    print(f"   'Other' category ({other_value}) count: {other_count}")
    return train_set, val_set

FREQUENCY_THRESHOLD = 5.0 # 5% threshold
features_to_bin = ["Mother's occupation", "Father's occupation", "Previous qualification"]

for i, feature in enumerate(features_to_bin, 1):
    print(f"{i}. {feature} Binning")
    train_set_eng, val_set_eng = apply_frequency_binning(train_set_eng, val_set_eng, feature, threshold_pct=FREQUENCY_THRESHOLD)
    print()


*** Applying Frequency-Based Binning ***
1. Mother's occupation Binning
   Binned 24/30 categories to 'Other'
   'Other' category (-1) count: 315

2. Father's occupation Binning
   Binned 31/39 categories to 'Other'
   'Other' category (-1) count: 364

3. Previous qualification Binning
   Binned 15/17 categories to 'Other'
   'Other' category (-1) count: 273



In [53]:
print("\nPost-Binning Value Distribution\n")

binned_features = [
    'Age at enrollment',
    "Mother's occupation",
    "Father's occupation",
    'Previous qualification'
]

for feature in binned_features:
    print(f"{feature}:")
    value_counts = train_set_eng[feature].value_counts().sort_index()
    percentages = (value_counts / len(train_set_eng) * 100).round(2)
    
    distribution_df = pd.DataFrame({
        'Count': value_counts,
        'Percentage': percentages
    })
    print(distribution_df)
    print(f"Total unique values: {train_set_eng[feature].nunique()}\n")


Post-Binning Value Distribution

Age at enrollment:
                   Count  Percentage
Age at enrollment                   
0                    591       23.87
1                   1256       50.73
2                    629       25.40
Total unique values: 3

Mother's occupation:
                     Count  Percentage
Mother's occupation                   
-1                     315       12.72
 2                     179        7.23
 3                     184        7.43
 4                     474       19.14
 5                     300       12.12
 7                     166        6.70
 9                     858       34.65
Total unique values: 7

Father's occupation:
                     Count  Percentage
Father's occupation                   
-1                     364       14.70
 3                     220        8.89
 4                     229        9.25
 5                     286       11.55
 6                     141        5.69
 7                     361       14.58
 8       

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [ ]:
# CUSTOM TRANSFORMERS (for reproducibility)

class StandardScalerTransformer(BaseEstimator, TransformerMixin):
    """Standardize numerical features"""
    
    def __init__(self, numerical_cols):
        self.numerical_cols = numerical_cols
        self.scaler = StandardScaler()
        
    def fit(self, X, y=None):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        # Only fit on columns that exist
        self.cols_to_scale = [c for c in self.numerical_cols if c in X_df.columns]
        if self.cols_to_scale:
            self.scaler.fit(X_df[self.cols_to_scale])
        return self
    
    def transform(self, X):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X.copy()
        if self.cols_to_scale:
            X_df[self.cols_to_scale] = self.scaler.transform(X_df[self.cols_to_scale])
        return X_df


class EncodingTransformer(BaseEstimator, TransformerMixin):
    """Handle both One-Hot and Ordinal encoding"""
    
    def __init__(self, nominal_cols, ordinal_cols):
        self.nominal_cols = nominal_cols
        self.ordinal_cols = ordinal_cols
        self.ohe = None
        self.oe = None
        self.ohe_feature_names = None
        
    def fit(self, X, y=None):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        
        # Filter to existing columns
        self.existing_nominal = [c for c in self.nominal_cols if c in X_df.columns]
        self.existing_ordinal = [c for c in self.ordinal_cols if c in X_df.columns]
        
        # Fit One-Hot Encoder
        if self.existing_nominal:
            self.ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore', dtype=int)
            self.ohe.fit(X_df[self.existing_nominal])
            self.ohe_feature_names = self.ohe.get_feature_names_out(self.existing_nominal)
        
        # Fit Ordinal Encoder
        if self.existing_ordinal:
            self.oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=int)
            self.oe.fit(X_df[self.existing_ordinal])
        
        return self
    
    def transform(self, X):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X.copy()
        
        # Apply One-Hot Encoding
        if self.ohe is not None and self.existing_nominal:
            ohe_data = self.ohe.transform(X_df[self.existing_nominal])
            ohe_df = pd.DataFrame(
                ohe_data,
                columns=self.ohe_feature_names,
                index=X_df.index
            )
            X_df = pd.concat([X_df.drop(columns=self.existing_nominal), ohe_df], axis=1)
        
        # Apply Ordinal Encoding
        if self.oe is not None and self.existing_ordinal:
            X_df[self.existing_ordinal] = self.oe.transform(X_df[self.existing_ordinal])
        
        return X_df


class ImbalanceHandler(BaseEstimator, TransformerMixin):
    """Handle imbalanced dataset using SMOTETomek"""
    
    def __init__(self, sampling_ratio=0.7, random_state=42):
        self.sampling_ratio = sampling_ratio
        self.random_state = random_state
        self.smote_tomek = None
        self.label_encoder = LabelEncoder()
        
    def fit(self, X, y=None):
        if y is None:
            raise ValueError("y is required for ImbalanceHandler")
        
        # Encode labels
        y_encoded = self.label_encoder.fit_transform(y)
        
        # Calculate sampling strategy
        class_counts = pd.Series(y).value_counts()
        majority_class = class_counts.idxmax()
        majority_count = class_counts.max()
        
        sampling_strategy = {}
        for class_label in class_counts.index:
            if class_label != majority_class:
                target_count = int(majority_count * self.sampling_ratio)
                sampling_strategy[self.label_encoder.transform([class_label])[0]] = target_count
        
        # Initialize SMOTETomek
        self.smote_tomek = SMOTETomek(
            smote=SMOTE(sampling_strategy=sampling_strategy, random_state=self.random_state, k_neighbors=5),
            random_state=self.random_state
        )
        
        return self
    
    def fit_resample(self, X, y):
        """Custom method for resampling"""
        self.fit(X, y)
        y_encoded = self.label_encoder.transform(y)
        X_resampled, y_resampled = self.smote_tomek.fit_resample(X, y_encoded)
        y_final = self.label_encoder.inverse_transform(y_resampled)
        return X_resampled, y_final
    
    def transform(self, X):
        # For pipeline compatibility - doesn't resample in transform
        return X


class NormalizationTransformer(BaseEstimator, TransformerMixin):
    """Apply Yeo-Johnson transformation to highly skewed features"""
    
    def __init__(self, skewness_threshold=1.0, exclude_binary=True):
        self.exclude_binary = exclude_binary
        self.skewness_threshold = skewness_threshold
        self.power_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
        self.highly_skewed_cols = []
        
    def fit(self, X, y=None):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        
        # Identify numerical columns
        numerical_cols = X_df.select_dtypes(include=[np.number]).columns

        # Exclude binary features if requested
        if self.exclude_binary:
            numerical_cols = [col for col in numerical_cols if X_df[col].nunique() > 2]  # Skip binary
        
        # Calculate skewness
        skewness = X_df[numerical_cols].skew()
        self.highly_skewed_cols = skewness[abs(skewness) > self.skewness_threshold].index.tolist()
        
        # Fit transformer
        if self.highly_skewed_cols:
            self.power_transformer.fit(X_df[self.highly_skewed_cols])
        
        return self
    
    def transform(self, X):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X.copy()
        
        if self.highly_skewed_cols:
            X_df[self.highly_skewed_cols] = self.power_transformer.transform(X_df[self.highly_skewed_cols])
        
        return X_df


class DimensionalityReducer(BaseEstimator, TransformerMixin):
    """Reduce dimensionality using variance threshold, feature selection, and optionally PCA"""
    
    def __init__(self, variance_threshold=0.01, k_features=50, apply_pca=True, pca_variance=0.95):
        self.variance_threshold = variance_threshold
        self.k_features = k_features
        self.apply_pca = apply_pca
        self.pca_variance = pca_variance
        
        self.variance_selector = VarianceThreshold(threshold=variance_threshold)
        self.feature_selector = None
        self.pca = None
        self.label_encoder = LabelEncoder()
        
    def fit(self, X, y=None):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X.copy()
        
        # Step 1: Variance threshold
        self.variance_selector.fit(X_df)
        X_var_filtered = X_df.loc[:, self.variance_selector.get_support()]
        
        # Step 2: Feature selection using mutual information
        if y is not None:
            y_encoded = self.label_encoder.fit_transform(y)
            k = min(self.k_features, X_var_filtered.shape[1])
            self.feature_selector = SelectKBest(score_func=mutual_info_classif, k=k)
            self.feature_selector.fit(X_var_filtered, y_encoded)
            X_selected = X_var_filtered.loc[:, self.feature_selector.get_support()]
        else:
            X_selected = X_var_filtered
        
        # Step 3: PCA (optional)
        if self.apply_pca:
            # First fit full PCA to check variance
            pca_full = PCA()
            pca_full.fit(X_selected)
            cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
            n_components = np.argmax(cumulative_variance >= self.pca_variance) + 1
            
            # Only apply PCA if it reduces dimensions by >30%
            if n_components < X_selected.shape[1] * 0.7:
                self.pca = PCA(n_components=n_components, random_state=42)
                self.pca.fit(X_selected)
                self.n_components = n_components
            else:
                self.pca = None
        
        return self
    
    def transform(self, X):
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X.copy()
        
        # Step 1: Variance threshold
        X_var_filtered = X_df.loc[:, self.variance_selector.get_support()]
        
        # Step 2: Feature selection
        if self.feature_selector is not None:
            X_selected = X_var_filtered.loc[:, self.feature_selector.get_support()]
        else:
            X_selected = X_var_filtered
        
        # Step 3: PCA
        if self.pca is not None:
            X_pca = self.pca.transform(X_selected)
            pca_columns = [f'PC{i+1}' for i in range(self.n_components)]
            X_final = pd.DataFrame(X_pca, columns=pca_columns, index=X_df.index)
        else:
            X_final = X_selected
        
        return X_final

In [55]:
# MAIN PREPROCESSING PIPELINE

print("\n" + "="*70)
print("B. DATA PREPROCESSING")
print("="*70)

# Separate features and target from engineered data
X_train = train_set_eng.drop(['Student_ID', 'Target'], axis=1)
y_train = train_set_eng['Target']
X_val = val_set_eng.drop(['Student_ID', 'Target'], axis=1)
y_val = val_set_eng['Target']

print(f"\nInitial shapes:")
print(f"  X_train: {X_train.shape}")
print(f"  X_val: {X_val.shape}")


B. DATA PREPROCESSING

Initial shapes:
  X_train: (2476, 33)
  X_val: (620, 33)


### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [56]:
numerical_cols_to_scale = [
    'Previous qualification (grade)', 
    'Admission grade',
    'Curricular units 1st sem (evaluations)',
    'Curricular units 1st sem (without evaluations)',
    'Curricular units 2nd sem (credited)',
    'Curricular units 2nd sem (enrolled)',
    'Curricular units 2nd sem (evaluations)',
    'Curricular units 2nd sem (grade)',
    'Curricular units 2nd sem (without evaluations)',
    'Unemployment rate',
    'Inflation rate',
    'GDP',
    'Approval_rate_sem1',
    'Approval_rate_sem2',
    'Grade_improvement'
]

# Filter to only existing columns
numerical_cols_to_scale = [col for col in numerical_cols_to_scale if col in X_train.columns]

print(f"\nApplying Standardization (Z-score) to {len(numerical_cols_to_scale)} numerical features...")
print(f"Method: StandardScaler (mean=0, std=1)")

scaler_transformer = StandardScalerTransformer(numerical_cols_to_scale)
scaler_transformer.fit(X_train)

X_train = scaler_transformer.transform(X_train)
X_val = scaler_transformer.transform(X_val)

print(f"Standardization complete")
print(f"   Mean after scaling: {X_train[numerical_cols_to_scale].mean().mean():.6f}")
print(f"   Std after scaling: {X_train[numerical_cols_to_scale].std().mean():.6f}")


Applying Standardization (Z-score) to 15 numerical features...
Method: StandardScaler (mean=0, std=1)
Standardization complete
   Mean after scaling: 0.000000
   Std after scaling: 0.866842


### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [57]:
# Encoding Analysis
# - There are various encoding methods to be tested as the categorical features have different characteristics
# - The heuristic used here is:
# 1. Label Encoding: used for ordinal data with order among categories, for example education level
# 2. One Hot Encoding: used for nominal data without order among categories
# 3. Mean encoding: used for high cardinality features to avoid dimensionality problem -> too many categories to be handled

# After analysis of the feature descriptions, the list of nominal and ordinal features are as such:
# - Nominal features for One-Hot Encoding
nominal_features_not_binary = [
    'Marital status',
    'Application mode',
    'Course',
    "Mother's occupation",
    "Father's occupation"
]
# Note: Binary features (0/1), will not be encoded as it is unnecessary.
# These binary features are: Daytime/evening attendance, Displaced, Educational special needs, Debtor, Tuition fees up to date, Gender, Scholarship, International

# - Ordinal features for Ordinal Encoding
ordinal_features = [
    'Application order',
    'Previous qualification',
    "Mother's qualification",
    "Father's qualification",
    'Age at enrollment'
]

print(f"\nEncoding Strategy:")
print(f"  One-Hot Encoding: {len(nominal_features_not_binary)} nominal features")
print(f"  Ordinal Encoding: {len(ordinal_features)} ordinal features")
print(f"  Applying One-Hot Encoding for features: {nominal_features_not_binary}")
print(f"  Applying Label/Ordinal Encoding for features: {ordinal_features}")

# Create and fit encoder
encoder_transformer = EncodingTransformer(nominal_features_not_binary, ordinal_features)
encoder_transformer.fit(X_train)

# Transform both sets
X_train = encoder_transformer.transform(X_train)
X_val = encoder_transformer.transform(X_val)

print(f"\nEncoding complete")
print(f"   Features after encoding: {X_train.shape[1]}")
if encoder_transformer.ohe is not None:
    print(f"   One-Hot created {len(encoder_transformer.ohe_feature_names)} features")


Encoding Strategy:
  One-Hot Encoding: 5 nominal features
  Ordinal Encoding: 5 ordinal features
  Applying One-Hot Encoding for features: ['Marital status', 'Application mode', 'Course', "Mother's occupation", "Father's occupation"]
  Applying Label/Ordinal Encoding for features: ['Application order', 'Previous qualification', "Mother's qualification", "Father's qualification", 'Age at enrollment']

Encoding complete
   Features after encoding: 83
   One-Hot created 55 features


### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [58]:
print("\nOriginal class distribution:")
print(y_train.value_counts())
print(f"\nClass proportions:")
print(y_train.value_counts(normalize=True) * 100)

# Calculate imbalance ratio
class_counts = y_train.value_counts()
imbalance_ratio = class_counts.max() / class_counts.min()
print(f"\nImbalance ratio: {imbalance_ratio:.2f}:1")

print(f"\n>>> Applying SMOTETomek (SMOTE + Tomek Links) <<<")
print("Strategy: Balance minority classes to 70% of majority class")

# Create and apply imbalance handler
imbalance_handler = ImbalanceHandler(sampling_ratio=0.7, random_state=42)
X_train_resampled, y_train_resampled = imbalance_handler.fit_resample(X_train, y_train)

# Convert back to DataFrame/Series for consistency
X_train = pd.DataFrame(X_train_resampled, columns=X_train.columns)
y_train = pd.Series(y_train_resampled, name='Target')

print(f"\nResampling complete")
print(f"   Training set size: {len(X_train)} samples")
print(f"\nNew class distribution:")
print(y_train.value_counts())
print(f"\nNew class proportions:")
print(y_train.value_counts(normalize=True) * 100)


Original class distribution:
Target
Graduate    1236
Dropout      795
Enrolled     445
Name: count, dtype: int64

Class proportions:
Target
Graduate    49.919225
Dropout     32.108239
Enrolled    17.972536
Name: proportion, dtype: float64

Imbalance ratio: 2.78:1

>>> Applying SMOTETomek (SMOTE + Tomek Links) <<<
Strategy: Balance minority classes to 70% of majority class

Resampling complete
   Training set size: 2832 samples

New class distribution:
Target
Graduate    1178
Enrolled     841
Dropout      813
Name: count, dtype: int64

New class proportions:
Target
Graduate    41.596045
Enrolled    29.696328
Dropout     28.707627
Name: proportion, dtype: float64


### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

In [59]:
print("\nAnalyzing feature distributions for normalization needs...")

# Calculate skewness for numerical features after encoding
# Get only numerical columns that exist now
current_numerical_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Calculate skewness on current data
skewness = X_train[current_numerical_cols].skew().sort_values(ascending=False)
print(f"\nTop 5 most skewed features:")
print(skewness.head())

highly_skewed_count = (abs(skewness) > 1.0).sum()
print(f"\nFeatures with |skewness| > 1.0: {highly_skewed_count}")

print(f"\n>>> Applying Yeo-Johnson Transformation <<<")
print("Method: PowerTransformer (handles both positive and negative values)")
print("Target: Features with |skewness| > 1.0")

# Create and fit normalization transformer
normalizer = NormalizationTransformer(skewness_threshold=1.0)
normalizer.fit(X_train)

if normalizer.highly_skewed_cols:
    print(f"\nNormalizing {len(normalizer.highly_skewed_cols)} features:")
    print(f"  {normalizer.highly_skewed_cols[:5]}..." if len(normalizer.highly_skewed_cols) > 5 else f"  {normalizer.highly_skewed_cols}")
    
    # Get skewness before transformation (only for the columns that will be transformed)
    skewness_before = X_train[normalizer.highly_skewed_cols].skew()
    
    # Transform both sets
    X_train = normalizer.transform(X_train)
    X_val = normalizer.transform(X_val)
    
    # Check improvement - calculate skewness after transformation
    skewness_after = X_train[normalizer.highly_skewed_cols].skew()
    
    print(f"\nNormalization complete")
    print(f"   Average |skewness| reduced from {abs(skewness_before).mean():.2f} to {abs(skewness_after).mean():.2f}")
else:
    print("\nNo highly skewed features found - normalization not needed")


Analyzing feature distributions for normalization needs...

Top 5 most skewed features:
Marital status_3       53.216539
Marital status_6       53.216539
Application mode_27    53.216539
Application mode_2     30.691999
Application mode_10    21.667935
dtype: float64

Features with |skewness| > 1.0: 64

>>> Applying Yeo-Johnson Transformation <<<
Method: PowerTransformer (handles both positive and negative values)
Target: Features with |skewness| > 1.0

Normalizing 5 features:
  ['Application order', 'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (grade)', 'Approval_rate_sem1', 'Grade_improvement']

Normalization complete
   Average |skewness| reduced from 1.91 to 0.95


### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

In [60]:
print(f"\nCurrent feature count: {X_train.shape[1]}")

print("\n>>> Step 1: Removing Low-Variance Features <<<")
print(f"Threshold: variance < 0.01")

print("\n>>> Step 2: Statistical Feature Selection <<<")
print(f"Method: SelectKBest with mutual_info_classif")
print(f"Target: Keep top 50 features (or all if less)")

print("\n>>> Step 3: PCA Analysis (Optional) <<<")
print(f"Method: PCA with 95% variance explained")
print(f"Condition: Only apply if >30% reduction possible")

# Create and fit dimensionality reducer
dim_reducer = DimensionalityReducer(
    variance_threshold=0.01,
    k_features=50,
    apply_pca=True,
    pca_variance=0.95
)

dim_reducer.fit(X_train, y_train)

# Transform both sets
X_train = dim_reducer.transform(X_train)
X_val = dim_reducer.transform(X_val)

print(f"\nDimensionality reduction complete")
print(f"   Final feature count: {X_train.shape[1]}")

if dim_reducer.pca is not None:
    print(f"   PCA applied: {dim_reducer.n_components} components explain {dim_reducer.pca_variance*100}% variance")
else:
    print(f"   PCA not applied - feature set already efficient")


Current feature count: 83

>>> Step 1: Removing Low-Variance Features <<<
Threshold: variance < 0.01

>>> Step 2: Statistical Feature Selection <<<
Method: SelectKBest with mutual_info_classif
Target: Keep top 50 features (or all if less)

>>> Step 3: PCA Analysis (Optional) <<<
Method: PCA with 95% variance explained
Condition: Only apply if >30% reduction possible

Dimensionality reduction complete
   Final feature count: 6
   PCA applied: 6 components explain 95.0% variance


# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [61]:
# Store all transformers for later use on test set
preprocessing_pipeline = {
    'scaler': scaler_transformer,
    'encoder': encoder_transformer,
    'normalizer': normalizer,
    'dim_reducer': dim_reducer
}

# Display the pipeline components
print("\nPreprocessing Pipeline Components:")
print("  1. StandardScalerTransformer      - Feature Scaling")
print("  2. EncodingTransformer            - Feature Encoding (One-Hot + Ordinal)")
print("  3. ImbalanceHandler               - SMOTE-Tomek (training only)")
print("  4. NormalizationTransformer       - Yeo-Johnson transformation")
print("  5. DimensionalityReducer          - Variance + Feature Selection + PCA")

print("\n✓ All transformers are fitted and ready to use")
print("\nStored in 'preprocessing_pipeline' dictionary:")
for key in preprocessing_pipeline.keys():
    print(f"  - preprocessing_pipeline['{key}']")

# Function to apply full pipeline to new data
def apply_preprocessing_pipeline(df, is_test=False, feature_engineering_done=False):
    """
    Apply the complete preprocessing pipeline to new data
    
    Parameters:
    -----------
    df : DataFrame
        Input dataframe
    is_test : bool
        If True, expects no 'Target' column and skips SMOTE
    feature_engineering_done : bool
        If True, assumes feature engineering already applied
    
    Returns:
    --------
    X_processed : DataFrame/array
        Preprocessed features
    y : Series (only if is_test=False)
        Target variable
    """
    
    df_processed = df.copy()
    
    # Apply feature engineering if not done
    if not feature_engineering_done:
        print("\nApplying feature engineering...")
        
        # Interaction features
        df_processed = interaction_features(df_processed)
        
        # Frequency-based binning
        for col in ["Mother's occupation", "Father's occupation", "Previous qualification"]:
            if col in df_processed.columns:
                value_counts = df_processed[col].value_counts()
                min_count = len(df_processed) * 0.05
                categories_to_bin = value_counts[value_counts < min_count].index.tolist()
                df_processed[col] = df_processed[col].apply(
                    lambda x: -1 if x in categories_to_bin else x
                )
        
        # Age binning
        def bin_age(age_log):
            if age_log < np.log(20):
                return 0
            elif age_log <= np.log(25):
                return 1
            else:
                return 2
        
        if 'Age at enrollment' in df_processed.columns:
            df_processed['Age at enrollment'] = df_processed['Age at enrollment'].apply(bin_age)
    
    # Separate features and target
    if is_test:
        X = df_processed.drop(['Student_ID'], axis=1)
        y = None
    else:
        X = df_processed.drop(['Student_ID', 'Target'], axis=1)
        y = df_processed['Target']
    
    # Apply preprocessing transformations
    print("\nApplying preprocessing pipeline...")
    
    print("  Step 1: Scaling...")
    X = preprocessing_pipeline['scaler'].transform(X)
    
    print("  Step 2: Encoding...")
    X = preprocessing_pipeline['encoder'].transform(X)
    
    # SMOTE is only applied during training, not on test/validation
    if not is_test:
        print("  Step 3: Skipping imbalance handling (already applied to training)")
    
    print("  Step 4: Normalization...")
    X = preprocessing_pipeline['normalizer'].transform(X)
    
    print("  Step 5: Dimensionality reduction...")
    X = preprocessing_pipeline['dim_reducer'].transform(X)
    
    print(f"\n✓ Preprocessing complete - Shape: {X.shape}")
    
    if is_test:
        return X
    else:
        return X, y

print("\n✓ Pipeline function 'apply_preprocessing_pipeline()' created")
print("\nUsage examples:")
print("  # For test set:")
print("  X_test = apply_preprocessing_pipeline(df_test, is_test=True)")
print("\n  # For validation (if needed):")
print("  X_new, y_new = apply_preprocessing_pipeline(df_new, is_test=False)")

# Verify pipeline works on validation set
print("\n" + "="*70)
print("PIPELINE VERIFICATION")
print("="*70)

print("\n✓ Pipeline is ready to use!")
print("\nCurrent preprocessed data available:")
print(f"  X_train shape:  {X_train.shape}")
print(f"  y_train shape:  {y_train.shape}")
print(f"  X_val shape:    {X_val.shape}")
print(f"  y_val shape:    {y_val.shape}")


Preprocessing Pipeline Components:
  1. StandardScalerTransformer      - Feature Scaling
  2. EncodingTransformer            - Feature Encoding (One-Hot + Ordinal)
  3. ImbalanceHandler               - SMOTE-Tomek (training only)
  4. NormalizationTransformer       - Yeo-Johnson transformation
  5. DimensionalityReducer          - Variance + Feature Selection + PCA

✓ All transformers are fitted and ready to use

Stored in 'preprocessing_pipeline' dictionary:
  - preprocessing_pipeline['scaler']
  - preprocessing_pipeline['encoder']
  - preprocessing_pipeline['normalizer']
  - preprocessing_pipeline['dim_reducer']

✓ Pipeline function 'apply_preprocessing_pipeline()' created

Usage examples:
  # For test set:
  X_test = apply_preprocessing_pipeline(df_test, is_test=True)

  # For validation (if needed):
  X_new, y_new = apply_preprocessing_pipeline(df_new, is_test=False)

PIPELINE VERIFICATION

✓ Pipeline is ready to use!

Current preprocessed data available:
  X_train shape:  (2832, 

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. DTL

In [62]:
# Type your code here

## B. Logistic Regression

In [63]:
# Type your code here

## C. SVM

In [64]:
# Type your code here

## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [65]:
# Type your code here

## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [66]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`